# 要約 
このJupyter Notebookは、大規模言語モデル（LLM）を利用して、ユーザーからのプロンプトに対する2つの異なる応答モデルのどちらが好まれるかを予測する問題に取り組んでいます。競技はKaggleの「LMSYS - Chatbot Arena」コンペティションの一環であり、具体的にはユーザーの好みに基づく会話システムを改善するための機械学習モデルの開発が目指されています。

### 使用する手法とライブラリ
ノートブックでは、次のような手法とライブラリが使用されています：

1. **ライブラリのインポート**:
   - 主に`numpy`, `pandas`, `torch`, `transformers`, `sklearn`, `keras`などのライブラリを使用して、データの処理、モデルの構築、トレーニングを行っています。

2. **トークナイザー**:
   - `transformers` ライブラリの `AutoTokenizer` を用い、テキストデータをトークン化します。特に、Qwen2のトークナイザーを使用し、入力シーケンスの最大長やパディングトークンの設定を行います。

3. **データ処理**:
   - トレーニングデータの前処理として、プロンプトと応答のクリーンアップを行い、さらにテキストを結合してモデルの入力用データを準備します。

4. **モデルの構築**:
   - CNN（畳み込みニューラルネットワーク）とLSTM（長短期記憶ネットワーク）のハイブリッドモデルを作成します。このモデルは、テキストの埋め込みを用いて、アテンションマスクを考慮した上での分類を行います。

5. **トレーニングと評価**:
   - モデルを訓練する際に、早期停止やモデルチェックポイントを使用して最適なパフォーマンスを保証します。訓練データを用いてモデルをトレーニングし、バリデーションデータで評価します。

6. **予測**:
   - テストデータに対してモデルを適用し、それぞれの応答モデルの勝者を予測します。最終的には、結果をCSVファイルとして出力します。

ノートブックの結論部分では、さらなるデータの追加や異なるハイパーパラメータの調整によってモデルの最適化の余地があることが示唆されています。全体として、ユーザーの好みを反映した会話生成モデルの構築に向けた包括的なアプローチが取られています。

---


# 用語概説 
以下は、Jupyter Notebook内で使用されている専門用語の解説です。マイナーなものや実務経験のないと馴染みのない用語に焦点を当てています。

### 専門用語の解説

1. **トークナイザー (Tokenizer)**
   - テキストをモデルが理解できる形式に変換するためのプロセス。具体的には、文字列を単語やサブワードの単位（トークン）に分割し、それぞれに整数IDを割り当てる役割を持つ。

2. **EOSトークン (End of Sequence Token)**
   - シーケンス（テキスト）の終わりを示す特別なトークン。モデルが、入力が終了したことを認識できるようにするために使用される。

3. **アテンションマスク (Attention Mask)**
   - モデルが特定の入力部分に注意を集中できるようにするためのマスク。この値は、入力の中でパディングされている部分を無視するために使用される。通常は1（注意を向ける）または0（無視する）の値を持つ。

4. **トークンの長さ (Token Length)**
   - テキスト内のトークンの数。トークンの長さはモデルの入力制限に影響を与え、長すぎる場合は切り捨てられることがある。

5. **フィルター数 (Number of Filters)**
   - 畳み込み層において、フィルター（カーネル）の数。各フィルターは入力データから異なる特徴を抽出する役割を持つ。フィルター数が多いほど、モデルの表現力が増すが、計算負荷も増加する。

6. **カーネルサイズ (Kernel Size)**
   - 畳み込みに使用されるフィルターのサイズ。特に1次元畳み込み層（Convolution1D）において、入力シーケンスのいくつの連続した要素を同時に見るかを示す。

7. **GlobalMaxPooling1D**
   - 1次元での最大プーリング層。特定の特徴マップ内の最大値を抽出し、次の層に縮小された特徴を供給する。長さの異なるシーケンスを統一する際に用いられる。

8. **ドロップアウト (Dropout)**
   - 過学習を防ぐための正則化手法。トレーニング中にランダムにニューロン（隠れ層のユニット）を無効化することで、モデルの汎用性を向上させる。

9. **アーリーストッピング (Early Stopping)**
   - バリデーションデータの損失が一定期間改善されない場合にトレーニングを早期に終了させる技術。過学習を防ぐためによく用いられる。

10. **アダデルタ (Adadelta)**
    - 学習率を適応的に調整する最適化アルゴリズムの一つ。過去の勾配に基づいて学習率を調整することにより、より安定したトレーニングを実現する。

11. **アダマックス (Adamax)**
    - Adamアルゴリズムの変種で、L1正則化が存在する場合に有効。特に大きな勾配を含むデータセットに対して安定した性能を発揮することができる。

12. **LSTM (Long Short-Term Memory)**
    - 隠れ層内にメモリセルを持つ特殊なリカレントニューラルネットワーク（RNN）の形式。長期依存関係を持つシーケンスデータの学習に特化している。

13. **マルチチャネルモデル (Multi-channel Model)**
    - 異なるタイプのデータやフィルターを同時に処理するために設計されたモデル。例えば、異なるサイズの畳み込みフィルターを使用して異なる特徴を捉えることができる。

これらの用語は、Notebookの操作や理解に役立つ背景知識として、特に実務経験の少ない初心者にとって重要です。

---


# ライブラリのインポート



In [ ]:
import os
import gc
import re
from time import time
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import torch
import transformers
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, AutoModel
import torch.nn.functional as F
np.random.seed(1337)  # 乱数のシードを1337に設定します。

# トークナイザー



In [ ]:
tokenizer = AutoTokenizer.from_pretrained("qwen/Qwen2-7B-Instruct")
tokenizer.pad_token = tokenizer.eos_token  # パディングトークンをEOSトークンに設定
tokenizer.padding_side = 'right'  # パディングを右側に設定
tokenizer.add_eos_token = True  # EOSトークンを追加

# 推論時にオフラインでトークナイザーを読み込むために保存
tokenizer.save_pretrained('tokenizer')

In [ ]:
# トークンの長さを取得するユーティリティ関数
def get_token_lengths(texts):
    # テキストごとにトークン化し、input_idsを取得
    input_ids = tokenizer(texts.tolist(), return_tensors='np')['input_ids']
    # 各テキストのinput_idsの長さを返す
    return [len(t) for t in input_ids]

# トレーニングデータの準備



In [ ]:
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')  # トレーニングデータをCSVファイルから読み込む

def process(input_str):
    stripped_str = input_str.strip('[]')  # 文字列からブラケットを削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文を分割してクオートを削除
    return  ' '.join(sentences)  # 文をスペースで結合して返す

train.loc[:, 'prompt'] = train['prompt'].apply(process)  # プロンプトを処理
train.loc[:, 'response_a'] = train['response_a'].apply(process)  # モデルAの応答を処理
train.loc[:, 'response_b'] = train['response_b'].apply(process)  # モデルBの応答を処理

In [ ]:
train.head(5)  # データフレームの先頭5行を表示

In [ ]:
train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']
print(train['text'][4])  # 4番目のテキストを表示

In [ ]:
# トレーニングデータセットの50％のみを使用
train = train[:int(len(train) * 1)]

train.loc[:, 'token_count'] = get_token_lengths(train['text'])  # 各テキストのトークン数を取得

# モデル用のラベルを準備
train.loc[:, 'label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)

# データを表示
display(train.head())

In [ ]:
train.label.value_counts()  # 各ラベルのカウントを表示

In [ ]:
# トークンのカウントの要約表示
display(train['token_count'].describe().to_frame().astype(int))

In [ ]:
# データの90%をカバーするトークンの長さを取得。ここでは1024の長さを使用します！
np.percentile(train['token_count'], 90)

# トークン化



In [ ]:
# データをトークン化
tokens = tokenizer(
    train['text'].tolist(), 
    max_length=1024, 
    truncation=True, 
    return_tensors='np')  # numpy配列として返す

# INPUT_IDSはトークンIDです
INPUT_IDS = tokens['input_ids']
# パディングトークンを無視するためのアテンションマスク
ATTENTION_MASKS = tokens['attention_mask']
# テキストのラベル
LABELS = train[['winner_model_a','winner_model_b','winner_tie']].values

print(f'INPUT_IDSの形状: {INPUT_IDS.shape}, ATTENTION_MASKSの形状: {ATTENTION_MASKS.shape}')  # 形状を表示
print(f'LABELSの形状: {LABELS.shape}')  # ラベルの形状を表示

In [ ]:
max_features = 14300  # 最大特徴量数
maxlen = 1024  # 最大トークン数
batch_size = 16  # バッチサイズ
embedding_dims = 100  # 埋め込み次元数
nb_filter = 150  # フィルター数
filter_length = 3  # フィルターの長さ
hidden_dims = 100  # 隠れ層の次元数
nb_epoch = 100  # エポック数

In [ ]:
from __future__ import print_function
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D, LSTM
from keras.datasets import imdb
from keras import backend as K
from keras.optimizers import Adadelta,Adamax
from keras.preprocessing import sequence as sq

from keras.layers import Dense, Dropout, Activation, Lambda,Input,TimeDistributed,Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint

from tensorflow.python.keras.backend import set_session as K
num_samples = INPUT_IDS.shape[0]  # サンプル数を取得

# X_validのサンプル数（X_trainの20%）
num_valid_samples = int(num_samples * 0.2)

# X_trainのインデックスをシャッフル
indices = np.random.permutation(num_samples)

# 最初の20%のインデックスをX_validのインデックスとして選択
valid_indices = indices[:num_valid_samples]

# 残りのインデックスをX_trainのインデックスとして選択
train_indices = indices[num_valid_samples:]

# 選ばれたインデックスから新しいX_validとX_trainを作成
X_train = sq.pad_sequences(INPUT_IDS[train_indices], maxlen=maxlen)  # パディングを適用
X_train_attention = sq.pad_sequences(ATTENTION_MASKS[train_indices], maxlen=maxlen)  # アテンションマスクにもパディングを適用
y_train = LABELS[train_indices]  # トレーニングラベル

X_valid = sq.pad_sequences(INPUT_IDS[valid_indices], maxlen=maxlen)  # バリデーションデータのパディング
X_valid_attention = sq.pad_sequences(ATTENTION_MASKS[valid_indices], maxlen=maxlen)  # バリデーションアテンションマスクのパディング
y_valid = LABELS[valid_indices]  # バリデーションラベル

In [ ]:
X_train = np.array(X_train)  # X_trainをnumpy配列に変換
y_train = np.array(y_train)  # y_trainをnumpy配列に変換
X_valid = np.array(X_valid)  # X_validをnumpy配列に変換
y_valid = np.array(y_valid)  # y_validをnumpy配列に変換

# モデルの定義



In [ ]:
'''この例は、テキスト分類におけるConvolution1Dの使用を示しています。
2エポック後にテスト精度は0.88に達します。
Intel i5 2.4GHz CPUで90秒/エポック。
Tesla K40 GPUで10秒/エポック。
'''
from tensorflow.keras.layers import Layer
from keras.layers import Concatenate
from keras.layers import  GlobalMaxPooling1D
import tensorflow as tf

#config = K.tf.ConfigProto(intra_op_parallelism_threads=16, inter_op_parallelism_threads=16, \
#                        allow_soft_placement=True, device_count = {'CPU': 1})

# tf_config = K.tf.ConfigProto()
# tf_config.gpu_options.allow_growth = True
# session = K.tf.Session(config=tf_config)
# K.set_session(session)

# config = K.tf.ConfigProto(intra_op_parallelism_threads=4, inter_op_parallelism_threads=4, \
#                         allow_soft_placement=True, device_count = {'CPU': 4})
# session = K.tf.Session(config=config)
# K.set_session(session)

class ApplyAttentionMask(Layer):
    def call(self, inputs):
        embeddings, attention_mask = inputs  # 入力から埋め込みとアテンションマスクを取得
        return embeddings * tf.expand_dims(attention_mask, -1)  # アテンションマスクを使用して埋め込みを適用

model = Sequential()

input_layer = Input(shape=(maxlen,),dtype='int32', name='main_input')  # 入力層を定義
attention_masks = Input(shape=(maxlen,), dtype='float32', name="attention_masks")  # アテンションマスクの入力層

emb_layer = Embedding(max_features,
                      embedding_dims,
                      input_length=maxlen
                      )(input_layer)  # 埋め込み層

masked_embeddings = ApplyAttentionMask(name='apply_attention_mask')([emb_layer, attention_masks])  # アテンションマスクを適用

def max_1d(X):
    return K.max(X, axis=1)  # 1Dの最大値を取得する関数

# 3のサイズのフィルターを持つConvolution1Dを追加
con3_layer = Convolution1D(filters=nb_filter,
                    padding='valid',
                    activation='relu',
                    kernel_size =3,
                    strides=1)(masked_embeddings)

pool_con3_layer = GlobalMaxPooling1D()(con3_layer)  # 最大プーリング

# 4のサイズのフィルターを持つConvolution1Dを追加
con4_layer = Convolution1D(filters=nb_filter,
                    kernel_size=5,
                    padding='valid',
                    activation='relu',
                    strides=1)(masked_embeddings)

pool_con4_layer = GlobalMaxPooling1D()(con4_layer)  # 最大プーリング

# 5のサイズのフィルターを持つConvolution1Dを追加
con5_layer = Convolution1D(filters=nb_filter,
                    kernel_size=7,
                    padding='valid',
                    activation='relu',
                    strides=1)(masked_embeddings)

pool_con5_layer = GlobalMaxPooling1D()(con5_layer)  # 最大プーリング

cnn_layer = Concatenate()([pool_con3_layer, pool_con5_layer, pool_con4_layer])  # CNNの出力を結合

# LSTM層を追加
x = masked_embeddings
lstm_layer = LSTM(128)(x)

cnn_lstm_layer = Concatenate()([lstm_layer, cnn_layer])  # LSTMとCNNの出力を結合

dense_layer = Dense(hidden_dims*2, activation='sigmoid')(cnn_lstm_layer)  # 隠れ層を追加
output_layer= Dropout(0.2)(dense_layer)  # ドロップアウトを追加
output_layer = Dense(3, trainable=True, activation='softmax')(output_layer)  # 出力層を定義

model = Model(inputs=[input_layer, attention_masks], outputs=[output_layer])  # モデルを定義
adadelta = Adadelta(learning_rate=1.0, rho=0.75, epsilon=1e-06)  # 最適化アルゴリズムを定義
adamax = Adamax(learning_rate=0.001)  # 別の最適化アルゴリズムを定義
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])  # モデルをコンパイル
model.summary()  # モデルの概要を表示

# モデルのトレーニング



In [ ]:
from keras.callbacks import EarlyStopping
checkpoint = ModelCheckpoint('CNN-LSTM-weights/weights.keras',
                                 monitor='val_acc', verbose=0, save_best_only=True,
                                 mode='max')  # 最良のモデルを保存するためのチェックポイント
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)  # アーリーストッピングの設定

model.fit([X_train,X_train_attention], y_train,
          batch_size=16,
          epochs=nb_epoch,
          callbacks=[checkpoint, early_stopping],
          validation_data=([X_valid,X_valid_attention], y_valid))  # モデルのトレーニング

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])  # モデルを再コンパイル
model.save('model_LSTM_mix_CNN.keras')  # モデルの保存

# モデルのテスト



In [ ]:
test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータをCSVファイルから読み込む

test.loc[:, 'prompt'] = test['prompt'].apply(process)  # プロンプトを処理
test.loc[:, 'response_a'] = test['response_a'].apply(process)  # モデルAの応答を処理
test.loc[:, 'response_b'] = test['response_b'].apply(process)  # モデルBの応答を処理

# トレーニング用に'Null'を削除
indexes = test[(test.response_a == 'null') & (test.response_b == 'null')].index
test.drop(indexes, inplace=True)  # 'Null'行を削除
test.reset_index(inplace=True, drop=True)  # インデックスをリセット

print(f"合計 {len(indexes)} 行のNull応答が削除されました")
print('合計トレーニングサンプル数: ', len(test))  # トレーニングサンプル数を表示

In [ ]:
test.head()  # テストデータの先頭5行を表示

In [ ]:
test['text'] = 'User prompt: ' + test['prompt'] +  '\n\nModel A :\n' + test['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']
print(test['text'])  # テキストを表示

In [ ]:
# データをトークン化
tokens_test = tokenizer(
    test['text'].tolist(), 
    max_length=1024, 
    truncation=True, 
    return_tensors='np')  # numpy配列として返す

# INPUT_IDSはトークンIDです
INPUT_test = tokens_test['input_ids']
# パディングトークンを無視するためのアテンションマスク
ATTENTION_MASKS2 = tokens_test['attention_mask']

print(f'INPUT_IDSの形状: {INPUT_test.shape}, ATTENTION_MASKSの形状: {ATTENTION_MASKS2.shape}')  # 形状を表示

In [ ]:
X_test = sq.pad_sequences(INPUT_test, maxlen=maxlen)  # テストデータにパディングを適用
X_test_attention = sq.pad_sequences(ATTENTION_MASKS2, maxlen=maxlen)  # テストアテンションマスクにもパディングを適用

In [ ]:
test  # テストデータを表示

In [ ]:
y_predict = model.predict([X_test,X_test_attention])  # テストデータで予測を実行
y_predict  # 予測結果を表示

In [ ]:
winner_df = pd.DataFrame(y_predict, columns=['winner_model_a', 'winner_model_b', 'winner_tie'])  # 予測結果をデータフレームに変換
result_df = pd.concat([test['id'], winner_df], axis=1)  # 結果データフレームを作成

In [ ]:
result_df.to_csv('submission.csv', index=False)  # 結果をCSVファイルに保存

In [ ]:
result_df  # 予測結果データフレームを表示

# 結論

トレーニングの速度を上げて最適化する余地はまだまだあります！ もっと多くのデータ、異なるバッチサイズ、学習率を試してみてください。ご健闘を祈ります！



---

# コメント

> ## carvingfate
> 
> Qwen2はGemma2よりも優れているのでしょうか？
> 
> > ## Nguyễn Anh Tú（トピック作成者）
> > 
> > Gemmaにアクセスできなかったため、Qwen 2を使用しています :(( ですので、レビューをお届けすることはできません。異なるトークナイザーを試して、モデルのハイパーパラメータを同じに保つことでジャッジしてください。私のモデルの構造を構築する際に参照した論文です：[https://www.researchgate.net/publication/321259272_Multi-channel_LSTM-CNN_model_for_Vietnamese_sentiment_analysis](https://www.researchgate.net/publication/321259272_Multi-channel_LSTM-CNN_model_for_Vietnamese_sentiment_analysis)
> > 
> > 

---